# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*.csv'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [2]:
file = 'event_datafile_new.csv'

In [22]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list), " rows total")
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
data_count = 0

with open(file, 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''): # this skips rows with empty 'artist'
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
        data_count += 1

print(data_count, " data rows")

8056  rows total
6820  data rows


In [23]:
# check the number of rows in your csv file
with open(file, 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f), " rows in event_datafile_new.csv, with 1 header row")

6821  rows in event_datafile_new.csv, with 1 header row


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [3]:
# This should make a connection to a Cassandra instance your local machine 
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e: print(e)

#### Create Keyspace

In [4]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e: print(e)

#### Set Keyspace

In [5]:
try: session.set_keyspace('udacity')
except Exception as e: print(e)

Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

**Query 1** Select artist, song title, song length from music_app_history where sessionId = 338 and itemInSession = 4

In [6]:
drop_1 = "DROP TABLE IF EXISTS music_app_history_session_item"
try: session.execute(drop_1)
except Exception as e: print(e)

create_1 = """
CREATE TABLE IF NOT EXISTS music_app_history_session_item (
    session_id int,
    item_in_session int,
    artist text,
    song text,
    length float,
    PRIMARY KEY (session_id, item_in_session)
)"""
try: session.execute(create_1)
except Exception as e: print(e)        

In [7]:
insert_1 = """
INSERT INTO music_app_history_session_item (
    session_id,
    item_in_session,
    artist,
    song,
    length)
VALUES (%s, %s, %s, %s, %s)"""

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            session.execute(insert_1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

Do a SELECT to verify that the data have been inserted into each table

In [9]:
select_1 = """
SELECT artist, song, length
FROM music_app_history_session_item
WHERE session_id = %s
AND item_in_session = %s
"""
try: rows = session.execute(select_1, (338, 4))
except Exception as e: print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


**Query 2** Select artist, song (sorted by itemInSession), user (first and last name) for userid = 10, sessionid = 182

We need `itemInSession` as a clustering column for the sort order and to complete a unique primary key.

In [10]:
drop_2 = "DROP TABLE IF EXISTS music_app_history_user_session_item"
try: session.execute(drop_2)
except Exception as e: print(e)

create_2 = """
CREATE TABLE IF NOT EXISTS music_app_history_user_session_item (
    user_id int,
    session_id int,
    item_in_session int,
    artist text,
    song text,
    first_name text,
    last_name text,
    PRIMARY KEY ((user_id, session_id), item_in_session)
) WITH CLUSTERING ORDER BY (item_in_session DESC)"""
try: session.execute(create_2)
except Exception as e: print(e)                

In [12]:
insert_2 = """
INSERT INTO music_app_history_user_session_item (
    user_id,
    session_id,
    item_in_session,
    artist,
    song,
    first_name,
    last_name)
VALUES (%s, %s, %s, %s, %s, %s, %s)"""

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            session.execute(insert_2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)

In [13]:
# To test the sort order, add ', item_in_session' into SELECT and ', row.item_in_session' into print()
select_2 = """
SELECT artist, song, first_name, last_name
FROM music_app_history_user_session_item
WHERE user_id = %s
AND session_id = %s
"""
try: rows = session.execute(select_2, (10, 182))
except Exception as e: print(e)
    
for row in rows:
    print(row.artist, row.song, row.first_name, row.last_name)

Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Down To The Bone Keep On Keepin' On Sylvie Cruz


**Query 3** Select user name (first and last) from music app history who listened to the song 'All Hands Against His Own'

The combination of song `title` and `userId` should make a sufficient primary key for this particular query. A user may listen to the same song multiple times within the same or different sessions. We can include `sessionId` and `itemInSession` in the primary key to foresee counts of times listened.

In [14]:
drop_3 = "DROP TABLE IF EXISTS music_app_history_song_user"
try: session.execute(drop_3)
except Exception as e: print(e)

create_3 = """
CREATE TABLE IF NOT EXISTS music_app_history_song_user (
    song text,
    user_id int,
    first_name text,
    last_name text,
    PRIMARY KEY (song, user_id)
)"""
try: session.execute(create_3)
except Exception as e: print(e)           

In [15]:
insert_3 = """
INSERT INTO music_app_history_song_user (
    song,
    user_id,
    first_name,
    last_name)
VALUES (%s, %s, %s, %s)"""

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            session.execute(insert_3, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)

In [16]:
select_3 = """
SELECT first_name, last_name
FROM music_app_history_song_user
WHERE song = %s
"""
try: rows = session.execute(select_3, ("All Hands Against His Own",)) # ,) https://stackoverflow.com/a/38090766
except Exception as e: print(e)
    
for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
try:
    session.execute(drop_1)
    session.execute(drop_2)
    session.execute(drop_3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()